In [1]:
import elasticsearch
import datetime
import pandas
import json
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# setup some initial vars
ES_HOST = {"host" : "128.173.145.158", "port" : 9200}
INDEX_NAME = 'dst'
TYPE_NAME = 'dst'
inpDstFile = "../dst_out_file.csv"
# request_body = {
#     "settings" : {
#         "number_of_shards": 1,
#         "number_of_replicas": 0
#     },
#     "mappings" : {
#         "_default_":{
#             "_timestamp":{
#                  "enabled":"true",
#                  "store":"true",
#                  "path":"plugins.time_stamp.string",
#                  "format":"yyyy-MM-dd HH:mm:ss"
#              }
#          }
#     }
# }
request_body = {
    "settings" : {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },

    'mappings': {
        TYPE_NAME: {
            'properties': {
                'dst_index': {'type': 'float'},
                'date': {'type': 'string'},
           }}}
}

In [3]:
# We'll put input the data as an array of jsons
dstRecs = [] 
# read data into a pandas DF
dstDF = pandas.read_csv(inpDstFile, sep=' ')
# Iterate over rows and create dict
# to upload to elasticsearch
for index, row in dstDF.iterrows():
    dstDict = {}
    dstDict["dst_index"] = row["dst_index"]
    dstDict["date"] = row["dst_date"]
    op_dict = {
        "index": {
            "_index": INDEX_NAME, 
            "_type": TYPE_NAME, 
            "_id": index
        }
    }
    dstRecs.append(op_dict)
    dstRecs.append(dstDict)


# dstRecsStr = dstDF.to_json(orient="records")
# dstRecs = json.loads(dstRecsStr)

In [4]:
es = elasticsearch.Elasticsearch(hosts = [ES_HOST], \
            timeout=30, max_retries=10, retry_on_timeout=True)
if es.indices.exists(INDEX_NAME):
    print "DELETING EXISTING INDEX--->", INDEX_NAME
    res = es.indices.delete(index=INDEX_NAME)
    print res
res = es.indices.create( index = INDEX_NAME, body = request_body )
print res

DELETING EXISTING INDEX---> dst
{u'acknowledged': True}
{u'acknowledged': True, u'shards_acknowledged': True}


In [5]:
res = es.bulk(index = INDEX_NAME, body = dstRecs, refresh = True)
print res

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.
